In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import six
project_path = os.getcwd() + '/../'
sys.path.append(project_path + "utils")

import sklearn.metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.grid_search import GridSearchCV
import sklearn
from sklearn.externals import joblib
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns

import function
import preprocessing
import local_bayes
import visualize
import metrics

In [3]:
sys.path.append(project_path + "models/cnn")
import cnn

# nearest points

In [4]:
save_data_dir = '../data/bookmark/'
train_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_train.npz')
test_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_test.npz')

In [5]:
new_tr = train_0_0['x'][:1000]
new_te = test_0_0['x'][0]

In [6]:
localbayes = local_bayes.LocalBayes(nn_num=20, data_dim=30)

In [7]:
nn_dist_ls, nn_ts_ls = localbayes.get_nearest_n(train_0_0['x'], new_te)

296

In [8]:
len(nn_ts_ls)

20

In [9]:
numpy.array(nn_ts_ls)

array([[ 7,  7,  5,  2,  1,  8,  8,  5,  3,  1,  2,  2,  2,  2,  5,  2,  3,
         4,  2,  4,  2,  3,  2,  5,  0,  1,  3,  1,  1,  1],
       [ 1,  5,  2,  2,  3,  7,  3,  2,  1,  1,  2,  3,  2,  2,  1,  3,  4,
         0,  3,  0,  3,  0,  1,  1,  0,  4,  1,  1,  0,  3],
       [ 1,  5,  2,  8,  5,  3,  1,  2,  3,  0,  3,  4,  1,  2,  1,  3,  1,
         5,  6,  1,  1,  1,  1,  2,  3,  2,  5,  0,  0,  1],
       [15,  9,  6,  5,  6,  6,  2,  2,  0,  3,  3,  4,  4,  4,  3,  6,  1,
         0,  0,  3,  2,  1,  1,  1,  3,  2,  3,  2,  1,  1],
       [ 6,  2,  7,  3,  2,  1,  1,  2,  2,  3,  2,  0,  1,  2,  3,  2,  3,
         2,  4,  0,  1,  3,  0,  4,  1,  7,  3,  1,  3,  5],
       [ 2,  4,  3,  3,  4,  3,  3,  2,  2,  2,  2,  0,  2,  1,  2,  1,  0,
         2,  6,  1,  4,  2,  2,  2,  1,  2,  2,  2,  1,  2],
       [ 5,  3,  2,  1,  1,  4,  2,  1,  4,  5,  3,  2,  4,  2,  5,  2,  2,
         1,  3,  3,  5,  1,  3,  1,  1,  1,  3,  4,  2,  3],
       [10,  6,  3,  0,  1,  3,  1,  4,  

## prediction

### gradient boosting decision tree

In [10]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
max_depth = 4
n_estimators = 5000

In [12]:
bdt = joblib.load('../models/gbdt/results/bdt_%s_%s.pkl' % (max_depth, n_estimators))

In [13]:
gbdt_pred = bdt.predict_proba(numpy.array(nn_ts_ls))
gbdt_pred = gbdt_pred[:, 1].tolist()
gbdt_pred

[0.5022338609488471,
 0.49973291447093166,
 0.5018741838472378,
 0.4940985358790975,
 0.5015701998906091,
 0.501042683257283,
 0.5008645755455798,
 0.5019696955333629,
 0.5010688578274138,
 0.502235012591178,
 0.4964808177388978,
 0.4974640891506576,
 0.5028709989644301,
 0.4930356207376856,
 0.4967185141986678,
 0.5024245986888581,
 0.5014492827008376,
 0.4834579565387829,
 0.4929238787186107,
 0.500018561432291]

### CNN

In [14]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import cuda
from chainer import optimizers
from chainer import serializers
from chainer import Link, Chain, ChainList

import pandas as pd
from ggplot import *

In [15]:
save_model_dir = '../models/cnn/results/awesome_100/'

In [16]:
sys.path.append(project_path + "models/cnn")
import cnn_net

In [17]:
N_test = numpy.array(nn_ts_ls).shape[0]
data_dim = test_0_0['x'].shape[1]

In [18]:
model = L.Classifier(cnn_net.tsCNN_net(data_dim))

In [19]:
serializers.load_npz(save_model_dir + "mlp.model", model)

In [20]:
def forward(x_data, model):
    x = chainer.Variable(x_data)
    h = F.max_pooling_2d(F.relu(model.conv1(x)), 2)
    h = F.max_pooling_2d(F.relu(model.conv2(h)), 2)
    h = F.relu(model.l1(h))
    y = model.l2(h)

    return y

In [21]:
nn_x = numpy.array(nn_ts_ls).reshape((N_test, 1, 1, data_dim))

In [22]:
cnn_pred = []
for i in range(0, N_test):
    x = np.asarray(nn_x[i]).reshape((1, 1, 1, data_dim)).astype(numpy.float32)
    
    pred_ = F.softmax(model.predictor(chainer.Variable(x)))
    cnn_pred.append(pred_.data[0][1])

In [23]:
cnn_pred

[0.99998808,
 0.99992847,
 0.99847454,
 0.00068034005,
 0.99983895,
 0.99991691,
 0.99835289,
 0.99988604,
 0.99999654,
 0.99999881,
 1.7782388e-06,
 0.00083333434,
 0.99995387,
 6.9481149e-07,
 0.0016020698,
 0.99998116,
 0.97835082,
 2.3233448e-34,
 2.7620164e-05,
 0.99978036]

## local bayes estimation

In [24]:
learner_name_ls = ['cnn', 'gbdt']
pred_proba_ls = [cnn_pred, gbdt_pred]

In [25]:
w_dict = localbayes.local_bayes_estimation(pred_proba_ls, learner_name_ls)

In [26]:
w_dict

{'cnn': 1.0, 'gbdt': 0.0}

In [27]:
model_dict = {}
model_dict.update({'cnn':model})
model_dict.update({'gbdt':bdt})

In [28]:
lb_pred = localbayes.local_bayes_predict(new_te, learner_name_ls, w_dict, model_dict)

In [47]:
import pulp

In [59]:
learner_name_ls = ['cnn', 'gbdt']
pred_proba_ls = [cnn_pred, gbdt_pred]

In [86]:
problem = pulp.LpProblem('localbayes', pulp.LpMaximize) # 最大化する場合

In [87]:
var = pulp.LpVariable.dicts('w', (learner_name), 0, 1, 'Continuous')

In [88]:
var

{'cnn': w_cnn, 'gbdt': w_gbdt}

In [89]:
## objective
for j in six.moves.range(20):
    model_sum = 0
    
    for i, learner in enumerate(learner_name):
        model_sum += var[learner] * math.log(pred_proba_ls[i][j] + 0.000000001)
        
    problem += model_sum

##  constraint
problem_ = 0

for learner in learner_name:
    problem += var[learner] >= 0
    problem_ += var[learner]
    
problem += problem_ == 1

In [90]:
model_sum

LpAffineExpression([(w_cnn, -5.759023890876643),
                    (w_gbdt, -0.6993868385363117)])

In [91]:
problem

localbayes:
MAXIMIZE
-5.75902389088*w_cnn + -0.699386838536*w_gbdt + -0.0
SUBJECT TO
_C1: w_cnn >= 0

_C2: w_gbdt >= 0

_C3: w_cnn + w_gbdt = 1

VARIABLES
w_cnn <= 1 Continuous
w_gbdt <= 1 Continuous

In [92]:
status = problem.solve()

In [93]:
pulp.LpStatus[status]

'Optimal'

In [97]:
for learner in learner_name:
    print '%s: %d' % (learner, var[learner].value())

cnn: 0
gbdt: 1


In [67]:
for i in six.moves.range(20):
    cnn = pred_[i] + 0.0000001
    gbdt = pred_proba[i][1]
    model_sum = var[1] * math.log(cnn) + var[2] * math.log(gbdt)
    problem += model_sum
problem += var[1] >= 0
problem += var[2] >= 0
problem += var[1] + var[2] == 1

In [68]:
problem

localbayes:
MAXIMIZE
-16.1176042533*w_1 + -0.699505407252*w_2 + -0.0
SUBJECT TO
_C1: w_1 >= 0

_C2: w_2 >= 0

_C3: w_1 + w_2 = 1

VARIABLES
w_1 <= 1 Continuous
w_2 <= 1 Continuous

In [69]:
status = problem.solve()

In [71]:
pulp.LpStatus[status]

'Optimal'

In [72]:
var[1].value()

0.0

In [73]:
var[2].value()

1.0

### generate prediction of nn train point

In [127]:
n = 20
data_dim = len(nn_ts_ls[0])
nn_ts_array = numpy.array(nn_ts_ls).reshape((n, 1, 1, data_dim))
pred_cnn = []

for i in six.moves.range(n):
    x = np.asarray(nn_ts_array[i]).reshape((1, 1, 1, data_dim)).astype(numpy.float32)
    
    pred = F.softmax(model.predictor(chainer.Variable(x)))
    pred_cnn.append(pred.data[0][1])
    
pred_cnn = numpy.array(pred_cnn)
pred_gbdt = bdt.predict_proba(nn_ts_array.reshape(20, 30))[:, 1].astype(float32)

In [128]:
pred_cnn

array([  9.99988079e-01,   9.99928474e-01,   9.98474538e-01,
         6.80340047e-04,   9.99838948e-01,   9.99916911e-01,
         9.98352885e-01,   9.99886036e-01,   9.99996543e-01,
         9.99998808e-01,   1.77823881e-06,   8.33334343e-04,
         9.99953866e-01,   6.94811490e-07,   1.60206982e-03,
         9.99981165e-01,   9.78350818e-01,   2.32334484e-34,
         2.76201645e-05,   9.99780357e-01], dtype=float32)

In [129]:
pred_gbdt

array([ 0.50223386,  0.49973291,  0.50187421,  0.49409854,  0.50157022,
        0.50104266,  0.50086457,  0.5019697 ,  0.50106883,  0.502235  ,
        0.49648082,  0.49746409,  0.50287098,  0.49303561,  0.49671853,
        0.5024246 ,  0.50144929,  0.48345795,  0.49292389,  0.50001854], dtype=float32)

In [ ]:
learner_name = ['cnn', 'gbdt']

In [ ]:
problem = pulp.LpProblem('localbayes', pulp.LpMaximize)
var = pulp.LpVariable.dicts('w', (learner_name_ls), 0, 1, 'Continuous')

In [ ]:
for i in six.moves.range(20):
    cnn = pred_[i] + 0.0000001
    gbdt = pred_proba[i][1]
    model_sum = var[1] * math.log(cnn) + var[2] * math.log(gbdt)
    problem += model_sum

In [ ]:
for learner in learner_name:
    problem += var[learner] >= 0

In [ ]:
for learner in learner_name:
    problem += var[learner]
problem == 1